In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [18]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils

In [404]:
sequence = sorted([p for p in CELLS])
sequence

[(-3, 0),
 (-3, 1),
 (-3, 2),
 (-3, 3),
 (-2, -1),
 (-2, 0),
 (-2, 1),
 (-2, 2),
 (-2, 3),
 (-1, -2),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (-1, 2),
 (-1, 3),
 (0, -3),
 (0, -2),
 (0, -1),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, -1),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, -3),
 (2, -2),
 (2, -1),
 (2, 0),
 (2, 1),
 (3, -3),
 (3, -2),
 (3, -1),
 (3, 0)]

In [6]:
a = np.array([[1]*39])
b = [500]

In [429]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,activation='relu', input_shape=(41,)))
    #model.add(layers.Dense(200, activation='relu'))
    #model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [430]:
model = build_model()

In [324]:
try:
    model = load_model('VanGame/my_model.h5')
except:
    print("none model found")
    model = build_model()

In [348]:
!ls 'rec'

'[1win]1556603515.0caoredjiba.txt'  '[2zzz]1556603518.0caoredjiba.txt'
'[2zzz]1556603429.0caoredjiba.txt'  '[2zzz]1556603519.0caoredjiba.txt'
'[2zzz]1556603510.0caoredjiba.txt'  '[2zzz]1556603520.0caoredjiba.txt'
'[2zzz]1556603511.0caoredjiba.txt'  '[2zzz]1556603521.0caoredjiba.txt'
'[2zzz]1556603514.0caoredjiba.txt'  '[2zzz]1556603523.0caoredjiba.txt'
'[2zzz]1556603517.0caoredjiba.txt'


In [24]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [25]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [26]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [335]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    states, color = utils.load_l('rec/'+filename)
    
    train = []
    value = []
    
    for s in states:
        # print(s)
        board = []
        # add board
        for cord in sequence:
            board.append(s['board'][cord])
        # color 
        board.append(color)
        # utility = [heuristic diff, my pieces difference, danger pieces]
        board += s['utility']
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.95
    curr_decay = 0.95
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    if value_check:
        print(value)
    
    post_value = np.array(value)
    
    post_value = (post_value - post_value.mean()) / post_value.std()
    
    if value_check:
        print(post_value)
    
    return train, post_value.tolist()

In [337]:
t,v = PG_log('[2zzz]1556601000.0caoredjiba.txt', value_check=True)

[0, -10, 0, 0, 0, -70, 0, 0, 0, 60, 0, -10, 0, 0, 0, 40, 0, 0, -10, 0, -50, -10, -10, 0, 0, -510]
[-2.9623299623520385, -10.679312596407287, -2.3265004496166717, -7.569378445443013, -23.395961757406386, -68.6981736973896, 3.6314572518325927, 9.623486373083757, 24.227440769097115, 57.943703590491936, -4.672055923825497, -10.08449031555048, -0.17325128244987167, -0.3374967455289259, -0.624577486817708, -1.0980615075465536, -68.64126599542232, -108.91127375337882, -164.16629427851393, -220.76193716255068, -300.31883085372016, -323.5009015115731, -384.8968642187499, -437.26124999999996, -484.5, -510]
[ 0.65992178  0.61523674  0.66360353  0.63324475  0.54160124  0.27927967
  0.69810297  0.7327997   0.81736361  1.01259696  0.65002164  0.61868105
  0.67607188  0.67512082  0.67345849  0.67071679  0.2796092   0.04642651
 -0.27352659 -0.60124254 -1.06191516 -1.19615048 -1.55166259 -1.85487753
 -2.12841258 -2.27606983]


[0.6599217763189812,
 0.6152367409499848,
 0.6636035345414046,
 0.6332447531642175,
 0.5416012354057987,
 0.2792796728434449,
 0.6981029708160643,
 0.7327996969734564,
 0.8173636061668272,
 1.0125969594253514,
 0.6500216419422908,
 0.6186810476069748,
 0.6760718812717054,
 0.6751208211650652,
 0.6734584858085444,
 0.6707167859478302,
 0.2796091957665567,
 0.0464265120278436,
 -0.2735265918685164,
 -0.6012425430284716,
 -1.061915163017154,
 -1.1961504848256177,
 -1.5516625930136971,
 -1.8548775335767187,
 -2.1284125814880457,
 -2.2760698273241196]

In [330]:
!ls rec

'[1win]1556602598.0caoredjiba.txt'  '[2zzz]1556602594.0caoredjiba.txt'
'[1win]1556602745.0caoredjiba.txt'  '[2zzz]1556602746.0caoredjiba.txt'
'[2zzz]1556601000.0caoredjiba.txt'


In [331]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')


In [384]:
def save_model():
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('trained_model', 'wb') as f:
        pickle.dump(model.get_weights(), f)

In [351]:
def train(save=True, epochs):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            #print(len(train), len(value))
            #print(train[-1])
            train_set += train
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    if save:
        print("saved")
        save_model()   

In [352]:
train()

Epoch 1/2
323/323 [==============================] - 2s 6ms/step - loss: 0.7966
Epoch 2/2
323/323 [==============================] - 1s 3ms/step - loss: 0.5005
saved


In [434]:
save_model()

In [321]:
del model

In [433]:
model.predict(np.array([[1]*41,[0.5]*41]))

array([[0.],
       [0.]], dtype=float32)

In [52]:
def clean_log(logs):
    !rm '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/' *

In [53]:
clean_log(logfs)

rm: cannot remove '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/': Is a directory
rm: cannot remove '__pycache__': Is a directory


In [110]:
os.getcwd()

'/Users/xinyaoniu/Documents/COMP30024-AI/Project2'

In [146]:
!rm -v ./rec/*

removed './rec/[1win]1556549315.0caoredjiba.txt'
removed './rec/[1win]1556549321.0caoredjiba.txt'
removed './rec/[2zzz]1556549305.0caoredjiba.txt'
removed './rec/[2zzz]1556549307.0caoredjiba.txt'
removed './rec/[2zzz]1556549308.0caoredjiba.txt'
removed './rec/[2zzz]1556549309.0caoredjiba.txt'
removed './rec/[2zzz]1556549311.0caoredjiba.txt'


In [353]:
def full_train_cycle(n, epochs=2):
    for i in range(n):
        !python3 -m referee VanGame HardCode HardCode -l logtry -v 0
    train(epochs=epochs)
    !rm -v ./rec/*

In [435]:
for i in range(50):
    print("********** starting round", i, "**********")
    full_train_cycle(2, epochs=1)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
63/63 [==============================] - 1s 15ms/step - loss: 1.6285
saved
removed './rec/[2zzz]1556609620.0caoredjiba.txt'
removed './rec/[2zzz]1556609622.0caoredjiba.txt'
********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
64/64 [==============================] - 0s 1ms/step - loss: 1.9555
saved
removed './rec/[1win]1556609625.0caoredjiba.txt'
removed './rec/[2zzz]1556609626.0caoredjiba.txt'
********** starting round 2 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1

********** starting round 21 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
105/105 [==============================] - 0s 1ms/step - loss: 0.9561
saved
removed './rec/[1win]1556609699.0caoredjiba.txt'
removed './rec/[2zzz]1556609696.0caoredjiba.txt'
********** starting round 22 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: game state occurred 4 times.
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
26/26 [==============================] - 0s 2ms/step - loss: 1.5116
saved
removed './rec/[2zzz]1556609702.0caoredjiba.txt'
********** starting round 23 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
80/80 [====

********** starting round 42 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
95/95 [==============================] - 0s 2ms/step - loss: 0.9936
saved
removed './rec/[1win]1556609784.0caoredjiba.txt'
removed './rec/[2zzz]1556609780.0caoredjiba.txt'
********** starting round 43 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
93/93 [==============================] - 0s 1ms/step - loss: 0.9144
saved
removed './rec/[1win]1556609788.0caoredjiba.txt'
removed './rec/[2zzz]1556609786.0caoredjiba.txt'
********** starting round 44 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1

In [440]:
# utility = [heuristic diff, my pieces difference, danger pieces]
model.get_weights()

[array([[-0.23024772],
        [-0.151471  ],
        [ 0.0569868 ],
        [-0.2190081 ],
        [ 0.04910022],
        [-0.2676637 ],
        [-0.07995524],
        [ 0.04810163],
        [-0.12606773],
        [ 0.19083132],
        [-0.14642501],
        [-0.39438084],
        [-0.43304476],
        [-0.4057517 ],
        [ 0.14259607],
        [-0.28709736],
        [-0.41580606],
        [-0.40851367],
        [-0.15074576],
        [-0.1016154 ],
        [ 0.1791379 ],
        [ 0.31065586],
        [-0.02695493],
        [-0.24903777],
        [-0.26632354],
        [ 0.16957459],
        [ 0.05476785],
        [ 0.28970903],
        [ 0.21947497],
        [ 0.1544554 ],
        [ 0.2754759 ],
        [-0.21634431],
        [ 0.24908917],
        [-0.29140475],
        [-0.03029402],
        [-0.3101741 ],
        [-0.3170652 ],
        [-0.09142641],
        [ 0.00571406],
        [ 0.17309135],
        [ 0.04784743]], dtype=float32),
 array([-0.1817355], dtype=float32),
 ar

# CNN attamp

In [226]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [227]:
for i in md.get_weights():
    print(i.shape)

(3, 3, 1, 1)


In [228]:
md.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 1)         9         
_________________________________________________________________
flatten_9 (Flatten)          (None, 676)               0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [229]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

array([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.

In [244]:
asdf = md.get_weights()[0].reshape(3,3)

In [230]:
wds = md.get_weights()

In [231]:
convp = wds[0]

In [232]:
convp.shape

(3, 3, 1, 1)

In [233]:
c = convp.reshape(1, 3, 3, 1)

In [234]:
d = c[0].reshape(3,3)

In [235]:
import scipy.signal as signal

In [236]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [245]:
e = signal.convolve2d(a, asdf, mode='valid')

In [246]:
e

array([[-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
      

In [206]:
signal.convolve(e, )

ValueError: volume and kernel should have the same dimensionality